## Challenge: Data cleaning & validation

In [1]:
import os
import glob
from datetime import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
#import taxi_functions as tf
import geopandas as gpd
from datetime import datetime
import matplotlib.dates as mdates
from scipy.stats import ttest_ind
%matplotlib inline

In [2]:
data = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = 'ISO-8859-1'); data.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [4]:
#drop na
data.dropna();
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [5]:
print('number of unique journal article is: ' + str(len(data['Journal title'].unique())))

number of unique journal article is: 985


In [6]:
#top 5 journal articles by name w/# of articles

data['Journal title'].value_counts()[:6]

PLoS One                                           92
PLoS ONE                                           62
Journal of Biological Chemistry                    48
Nucleic Acids Research                             21
Proceedings of the National Academy of Sciences    19
PLoS Neglected Tropical Diseases                   18
Name: Journal title, dtype: int64

In [8]:
#make all journal articles lower case

data['Journal title'] = data['Journal title'].apply(lambda x: str(x).lower())

In [9]:
#re-run the top five w/lower case values
top_five = data['Journal title'].value_counts()[:5]; top_five

plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos genetics                       24
plos pathogens                      24
Name: Journal title, dtype: int64

In [10]:
#check to see if cost column can be used for calculations
data['COST (£) charged to Wellcome (inc VAT when charged)'].apply(lambda x: str(x).isdigit()).head()

0    False
1    False
2    False
3    False
4    False
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: bool

In [20]:
#make a new dataframe with values of lower case and price data so we can manipulate the price data

journal_df = data[['Journal title', 'COST (£) charged to Wellcome (inc VAT when charged)']]; journal_df.head()

,Journal title,COST (£) charged to Wellcome (inc VAT when charged)
0,psychological medicine,£0.00
1,biomacromolecules,£2381.04
2,j med chem,£642.56
3,j med chem,£669.64
4,j org chem,£685.88


In [22]:
#need to drop pound symbol and make numeric

drop_pound = journal_df['COST (£) charged to Wellcome (inc VAT when charged)'].str.split('£', expand=True); drop_pound.head()

,0,1
0,,0.00
1,,2381.04
2,,642.56
3,,669.64
4,,685.88


In [31]:
cleaned_df = pd.concat([journal_df['Journal title'], drop_pound], axis=1); cleaned_df.head()

,Journal title,0,1
0,psychological medicine,,0.00
1,biomacromolecules,,2381.04
2,j med chem,,642.56
3,j med chem,,669.64
4,j org chem,,685.88


In [59]:
cleaned_df[1] = cleaned_df[1].astype(float); cleaned_df.head()

,Journal title,0,1
0,psychological medicine,,0.00
1,biomacromolecules,,2381.04
2,j med chem,,642.56
3,j med chem,,669.64
4,j org chem,,685.88


In [61]:
#just a test to see if it works for the plos_one
plos_one = cleaned_df[cleaned_df['Journal title']=='plos one']
print(plos_one.mean())
print(plos_one.median())
print(plos_one.std())

1    38751.291368
dtype: float64
1    898.475
dtype: float64
1    189009.511974
dtype: float64


[1580.4]